# Request

In [1]:
# import re
# import requests
# from bs4 import BeautifulSoup

In [2]:
# def geocode_address(
#     address: str
# ) -> str:
#     """
#     Generate latitude and longitude from address using Google Maps.
#     Args:
#         address: Address to be geocoded.
#     """
#     # User agent and headers from: https://stackoverflow.com/a/51955893
#     user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
#     headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

#     address = address.replace(' ', '+')
#     r = requests.get(f'https://www.google.com.br/maps/place/{address}',headers=headers)
#     soup = BeautifulSoup(r.content, 'html.parser')
#     try:
#         full_content = ''
#         for item in soup.find_all('meta'):
#             full_content += item['content']

#         latlong = re.findall(r'-\d+.\d+,-\d+.\d+', full_content)[0]

#         time.sleep(2)    
#         return latlong
#     except:
#         return None

In [3]:
# # Bing
# user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
# headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

# address = 'RODOVIA 163 - KM 118 - ZONA RURAL. CEP:79950-000 - NAVIRAI/MS'
# r = requests.get(f'https://www.bing.com/maps?q={address}',headers=headers)
# soup = BeautifulSoup(r.content, 'html.parser')

# print(soup.prettify())

# Selenium

In [5]:
import os
import re
import time
import pandas as pd
import selenium.webdriver as webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from random import random

def selenium_setup() -> tuple:
    """
    Setup selenium webdriver for Firefox
    """

    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0'
    mozilla_driver = os.path.join('../data/01_raw', 'geckodriver.exe')
    mozilla_service = Service(mozilla_driver, log_output='../logs/geckodriver.log')
    mozilla_options = Options()
    mozilla_options.set_preference('general.useragent.override', user_agent)
    return  mozilla_service, mozilla_options

In [2]:
rf_renovabio_plants = catalog.load("refined_renovabio_plants").reset_index(drop=True)

                    INFO     Loading data from 'refined_renovabio_plants' (ParquetDataSet)...   ]8;id=64582;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=394841;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

In [3]:
def generate_address_lists(
    rf_renovabio_plants: pd.DataFrame,
) -> tuple(list, list):
    """
    Generate a list of addresses to be geocoded.
    """

    rf_renovabio_plants = rf_renovabio_plants.fillna('')

    address_list_v1 = (
        rf_renovabio_plants['DS_END'].str.replace('FAZENDA', 'USINA')
        .str.cat(
            [rf_renovabio_plants['NO_END'],
            rf_renovabio_plants['CIDADE'],
            rf_renovabio_plants['UF'],
            rf_renovabio_plants['CEP']],
            sep=' '
        )
    ).tolist()

    address_list_v2 = (
        rf_renovabio_plants['CIDADE']
        .str.cat(
            [rf_renovabio_plants['UF'],
            rf_renovabio_plants['CEP']],
            sep=' '
        )
    ).tolist()

    return address_list_v1, address_list_v2

In [7]:
def get_latlong_from_url(url: str) -> tuple:
    """
    Returns latitude and longitude from a Bing Maps URL
    """
    latlong = re.findall(r'-?\d{1,2}\.\d{3,}', url)
    latitude = float(latlong[0])
    longitude = float(latlong[1])
    return latitude, longitude


def selenium_geocode(
    mozilla_service,
    mozilla_options,
    address_list_v1: list,
    address_list_v2: list,
) -> pd.DataFrame:
    """
    Geocodes addresses using Selenium and Firefox
    """
    # Launch browser
    browser = webdriver.Firefox(service=mozilla_service, options=mozilla_options)

    # Geocoding addresses
    latitude = []
    longitude = []

    first_iter = True
    for i in range(len(address_list_v1)):
        browser.get(f'https://www.bing.com/maps?q={address_list_v1[i]}')

        # Longer sleep time for first iteration (browser opens)
        if first_iter is True:
            time.sleep(6)
            first_iter = False
        else:
            time.sleep(3.1 + random())

        lat, long = get_latlong_from_url(browser.current_url)

        # If address is not found, try to geocode only the last 3 address columns
        ckeck_results = re.findall('Não há resultados para:', browser.page_source)
        if len(ckeck_results) > 0:
            browser.get(f'https://www.bing.com/maps?q={address_list_v2[i]}')
            time.sleep(3.1)

            lat, long = get_latlong_from_url(browser.current_url)

        latitude.append(lat)
        longitude.append(long)

    browser.quit()

    df_latlong = pd.DataFrame({'LATITUDE': latitude, 'LONGITUDE': longitude})
    return df_latlong

In [8]:
def geocode_renovabio_plants(
    rf_renovabio_plants: pd.DataFrame,
):

    mozilla_service, mozilla_options = selenium_setup()
    address_list_v1, address_list_v2 = generate_address_lists(rf_renovabio_plants)

    df_latlong = selenium_geocode(
        mozilla_service, mozilla_options,
        address_list_v1, address_list_v2,
    )

    rf_renovabio_plants_geocoded = pd.concat([
        rf_renovabio_plants, df_latlong
    ], axis=1, ignore_index=False)

    return rf_renovabio_plants_geocoded